# 문제 6: 지하철역 승하차 인원 수집, 저장
(ds0_open_data)

문제 5에서 사용한 OPI를 가지고 진행

* 이용API : http://data.seoul.go.kr/dataList/OA-12252/S/1/datasetView.do
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-5번-문제-일괄-실행-(json파일-생성)" data-toc-modified-id="1.-5번-문제-일괄-실행-(json파일-생성)-1">1. 5번 문제 일괄 실행 (json파일 생성)</a></span></li><li><span><a href="#2.-json파일이-잘-저장됐는지-확인" data-toc-modified-id="2.-json파일이-잘-저장됐는지-확인-2">2. json파일이 잘 저장됐는지 확인</a></span></li><li><span><a href="#3.-pymongo-결과-확인" data-toc-modified-id="3.-pymongo-결과-확인-3">3. pymongo 결과 확인</a></span><ul class="toc-item"><li><span><a href="#1)-데이터베이스-생성-확인" data-toc-modified-id="1)-데이터베이스-생성-확인-3.1">1) 데이터베이스 생성 확인</a></span></li><li><span><a href="#2)-테이블-생성-확인" data-toc-modified-id="2)-테이블-생성-확인-3.2">2) 테이블 생성 확인</a></span></li><li><span><a href="#3)-몇건-저장됐는지-확인" data-toc-modified-id="3)-몇건-저장됐는지-확인-3.3">3) 몇건 저장됐는지 확인</a></span></li><li><span><a href="#4)-1건만-출력해보기" data-toc-modified-id="4)-1건만-출력해보기-3.4">4) 1건만 출력해보기</a></span></li></ul></li><li><span><a href="#4.-MongoDB-cli-결과-확인" data-toc-modified-id="4.-MongoDB-cli-결과-확인-4">4. MongoDB cli 결과 확인</a></span><ul class="toc-item"><li><span><a href="#1)-cmd-경로-이동" data-toc-modified-id="1)-cmd-경로-이동-4.1">1) cmd 경로 이동</a></span></li><li><span><a href="#2)-DB-존재하는지-확인" data-toc-modified-id="2)-DB-존재하는지-확인-4.2">2) DB 존재하는지 확인</a></span></li><li><span><a href="#3)-DB를-쉬위치" data-toc-modified-id="3)-DB를-쉬위치-4.3">3) DB를 쉬위치</a></span></li><li><span><a href="#4)-테이블-존재-확인" data-toc-modified-id="4)-테이블-존재-확인-4.4">4) 테이블 존재 확인</a></span></li><li><span><a href="#5)-전체-조회" data-toc-modified-id="5)-전체-조회-4.5">5) 전체 조회</a></span></li><li><span><a href="#6)-특정항목-조회" data-toc-modified-id="6)-특정항목-조회-4.6">6) 특정항목 조회</a></span></li></ul></li></ul></div>

---
### 1. 5번 문제 일괄 실행 (json파일 생성)

In [1]:
%%writefile src/ds_open_subwayTime.py
# coding: utf-8
import os
import requests
import json
from pymongo import MongoClient
import mylib

Client = MongoClient('localhost:27017') # 클라이언트 만들기
_db=Client['ds_open_subwayPassengersDb'] #db created by mongo. You do not have to create this. db명 
_table=_db['db_open_subwayTable'] #collection table 명
#db=Client.ds_rest_subwayPassengers

def saveJson(_fname,_data):
    import io
    with io.open(_fname, 'a', encoding='utf8') as json_file: # a(append) = 붙여넣기 추가
        #_j=json.dumps(_data, json_file, ensure_ascii=False, encoding='utf8') # python2
        _j=json.dump(_data, json_file, ensure_ascii=False)
        json_file.write(str(_j)+"\n")

def readJson(_fname):
    for line in open(_fname, 'r').readlines():
        _j=json.loads(line)
        #print _j['id'],_j['text']
        print (_j['id'])

def saveDB(_data):
    _table.insert_one(_data)

def readDB():
    for tweet in _table.find():
        print (tweet['id'],tweet['text'])

def saveFile(_fname,_data):
    fp=open(_fname,'a')
    fp.write(_data+"\n")

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)  
    _key=key['02_05'] #KEY='73725.....'
    _url='http://openAPI.seoul.go.kr:8088'
    _type='json'
    _service='CardSubwayTime'
    _start_index=1
    _end_index=5
    _use_mon='202106'
    _maxIter=20
    _iter=0
    _jfname='src/ds_open_subwayTime.json'
    while _iter<_maxIter:
        _api="/".join([_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon])
        #print _api
        r=requests.get(_api)
        _json=r.json()
        print (_json)
        # 저장 
        saveJson(_jfname,_json)
        saveDB(_json)
        # /저장
        _start_index+=5
        _end_index+=5
        _iter+=1

if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_subwayTime.py


In [2]:
!python src/ds_open_subwayTime.py

{'CardSubwayTime': {'list_total_count': 608, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'USE_MON': '202106', 'LINE_NUM': '1호선', 'SUB_STA_NM': '서울역', 'FOUR_RIDE_NUM': 654.0, 'FOUR_ALIGHT_NUM': 17.0, 'FIVE_RIDE_NUM': 9008.0, 'FIVE_ALIGHT_NUM': 6400.0, 'SIX_RIDE_NUM': 12474.0, 'SIX_ALIGHT_NUM': 37203.0, 'SEVEN_RIDE_NUM': 37253.0, 'SEVEN_ALIGHT_NUM': 91875.0, 'EIGHT_RIDE_NUM': 59876.0, 'EIGHT_ALIGHT_NUM': 187805.0, 'NINE_RIDE_NUM': 44619.0, 'NINE_ALIGHT_NUM': 118679.0, 'TEN_RIDE_NUM': 42611.0, 'TEN_ALIGHT_NUM': 57710.0, 'ELEVEN_RIDE_NUM': 49533.0, 'ELEVEN_ALIGHT_NUM': 50003.0, 'TWELVE_RIDE_NUM': 59357.0, 'TWELVE_ALIGHT_NUM': 53317.0, 'THIRTEEN_RIDE_NUM': 61171.0, 'THIRTEEN_ALIGHT_NUM': 53687.0, 'FOURTEEN_RIDE_NUM': 53310.0, 'FOURTEEN_ALIGHT_NUM': 49094.0, 'FIFTEEN_RIDE_NUM': 65767.0, 'FIFTEEN_ALIGHT_NUM': 52788.0, 'SIXTEEN_RIDE_NUM': 76249.0, 'SIXTEEN_ALIGHT_NUM': 53969.0, 'SEVENTEEN_RIDE_NUM': 122928.0, 'SEVENTEEN_ALIGHT_NUM': 64693.0, 'EIGHTEEN_RIDE_NUM': 184907.0,

### 2. json파일이 잘 저장됐는지 확인

In [12]:
!dir -l src\ds_open_subwayTime*

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 4E88-AB80

 C:\Users\Nunu\BigData 디렉터리


 C:\Users\Nunu\BigData\src 디렉터리

2021-09-13  오후 11:44           445,692 ds_open_subwayTime.json
2021-09-13  오후 11:44             1,895 ds_open_subwayTime.py
               2개 파일             447,587 바이트
               0개 디렉터리  29,106,487,296 바이트 남음


파일을 찾을 수 없습니다.


### 3. pymongo 결과 확인

In [14]:
import pymongo

Client = pymongo.MongoClient('mongodb://localhost:27017') # or pymongo.MongoClient() without uri

#### 1) 데이터베이스 생성 확인

In [15]:
print(Client.list_database_names())

['admin', 'config', 'ds_open_subwayPassengersDb', 'local', 'myDB']


#### 2) 테이블 생성 확인

In [18]:
db=Client.ds_open_subwayPassengersDb
db.list_collection_names()

['db_open_subwayTable']

#### 3) 몇건 저장됐는지 확인
원래는 20건이지만 중복 실행으로 그 이상이 나옴

In [19]:
db.db_open_subwayTable.count_documents({})

60

#### 4) 1건만 출력해보기

In [20]:
db.db_open_subwayTable.find_one()

{'_id': ObjectId('613da120e8c5b9dae89c627a'),
 'CardSubwayTime': {'list_total_count': 608,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'USE_MON': '202106',
    'LINE_NUM': '1호선',
    'SUB_STA_NM': '서울역',
    'FOUR_RIDE_NUM': 654.0,
    'FOUR_ALIGHT_NUM': 17.0,
    'FIVE_RIDE_NUM': 9008.0,
    'FIVE_ALIGHT_NUM': 6400.0,
    'SIX_RIDE_NUM': 12474.0,
    'SIX_ALIGHT_NUM': 37203.0,
    'SEVEN_RIDE_NUM': 37253.0,
    'SEVEN_ALIGHT_NUM': 91875.0,
    'EIGHT_RIDE_NUM': 59876.0,
    'EIGHT_ALIGHT_NUM': 187805.0,
    'NINE_RIDE_NUM': 44619.0,
    'NINE_ALIGHT_NUM': 118679.0,
    'TEN_RIDE_NUM': 42611.0,
    'TEN_ALIGHT_NUM': 57710.0,
    'ELEVEN_RIDE_NUM': 49533.0,
    'ELEVEN_ALIGHT_NUM': 50003.0,
    'TWELVE_RIDE_NUM': 59357.0,
    'TWELVE_ALIGHT_NUM': 53317.0,
    'THIRTEEN_RIDE_NUM': 61171.0,
    'THIRTEEN_ALIGHT_NUM': 53687.0,
    'FOURTEEN_RIDE_NUM': 53310.0,
    'FOURTEEN_ALIGHT_NUM': 49094.0,
    'FIFTEEN_RIDE_NUM': 65767.0,
    'FIFTEEN_ALIGHT_NUM': 52788.0,
 

In [21]:
db.db_open_subwayTable.find_one({"CardSubwayTime.row.SUB_STA_NM":"서울역"})

{'_id': ObjectId('613da120e8c5b9dae89c627a'),
 'CardSubwayTime': {'list_total_count': 608,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'USE_MON': '202106',
    'LINE_NUM': '1호선',
    'SUB_STA_NM': '서울역',
    'FOUR_RIDE_NUM': 654.0,
    'FOUR_ALIGHT_NUM': 17.0,
    'FIVE_RIDE_NUM': 9008.0,
    'FIVE_ALIGHT_NUM': 6400.0,
    'SIX_RIDE_NUM': 12474.0,
    'SIX_ALIGHT_NUM': 37203.0,
    'SEVEN_RIDE_NUM': 37253.0,
    'SEVEN_ALIGHT_NUM': 91875.0,
    'EIGHT_RIDE_NUM': 59876.0,
    'EIGHT_ALIGHT_NUM': 187805.0,
    'NINE_RIDE_NUM': 44619.0,
    'NINE_ALIGHT_NUM': 118679.0,
    'TEN_RIDE_NUM': 42611.0,
    'TEN_ALIGHT_NUM': 57710.0,
    'ELEVEN_RIDE_NUM': 49533.0,
    'ELEVEN_ALIGHT_NUM': 50003.0,
    'TWELVE_RIDE_NUM': 59357.0,
    'TWELVE_ALIGHT_NUM': 53317.0,
    'THIRTEEN_RIDE_NUM': 61171.0,
    'THIRTEEN_ALIGHT_NUM': 53687.0,
    'FOURTEEN_RIDE_NUM': 53310.0,
    'FOURTEEN_ALIGHT_NUM': 49094.0,
    'FIFTEEN_RIDE_NUM': 65767.0,
    'FIFTEEN_ALIGHT_NUM': 52788.0,
 

### 4. MongoDB cli 결과 확인

#### 1) cmd 경로 이동

In [22]:
import os
os.getcwd() # 홈디렉토리

'C:\\Users\\Nunu\\BigData'

`mongo` 입력

#### 2) DB 존재하는지 확인
`> show dbs` 입력

#### 3) DB를 쉬위치
`> use ds_open_subwayPassengersDb` 입력

#### 4) 테이블 존재 확인
`> show collections` 입력

#### 5) 전체 조회
`> db.db_open_subwayTable.find()` 입력

#### 6) 특정항목 조회
`> db.db_open_subwayTable.find({"CardSubwayTime.row.SUB_STA_NM":"서울역"})` 입력

In [23]:
%%writefile src/ds_open_subwayTime_mongo.js
show dbs
use ds_open_subwayPassengersDb
show collections
db.db_open_subwayTable.find()
db.db_open_subwayTable.find({"CardSubwayTime.row.SUB_STA_NM":"서울역"})

Writing src/ds_open_subwayTime_mongo.js


In [24]:
!mongo < src/ds_open_subwayTime_mongo.js

MongoDB shell version v5.0.2
connecting to: mongodb://127.0.0.1:27017/?compressors=disabled&gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("e4780f4c-e127-4700-93b5-592d4a085865") }
MongoDB server version: 5.0.2
which delivers improved usability and compatibility.The "mongo" shell has been deprecated and will be removed in
an upcoming release.
We recommend you begin using "mongosh".
For installation instructions, see
https://docs.mongodb.com/mongodb-shell/install/
admin                       0.000GB
config                      0.000GB
ds_open_subwayPassengersDb  0.000GB
local                       0.000GB
myDB                        0.000GB
switched to db ds_open_subwayPassengersDb
db_open_subwayTable
{ "_id" : ObjectId("613da120e8c5b9dae89c627a"), "CardSubwayTime" : { "list_total_count" : 608, "RESULT" : { "CODE" : "INFO-000", "MESSAGE" : "�젙�긽 泥섎━�릺�뿀�뒿�땲�떎" }, "row" : [ { "USE_MON" : "202106", "LINE_NUM" : "1�샇�꽑", "SUB_STA_NM" : "�꽌�슱�뿭", "FOUR_RIDE_NUM" : 654, "F